In [5]:
!apt-get update && apt-get install -y wget ffmpeg libsm6 libxext6 && apt-get clean
!pip install opencv-python
!./download_dataset.sh
!python generate_labels.py

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages wi

In [6]:
import os
from glob import glob
import cv2
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, \
                                    MaxPool2D, Input, UpSampling2D, concatenate, Input

BATCH_SIZE = 8
MODEL_SHAPE = (256,  256)
EPOCHS = 200

In [7]:
def load_train_pairs():
    filenames = set([os.path.basename(file) for file in glob('dataset/unlabeled/*.png')] +
                    [os.path.basename(file) for file in glob('dataset/labeled/*.png')])
    unlabeled_paths = [os.path.join('dataset/unlabeled', file) for file in filenames]
    labeled_paths = [os.path.join('dataset/labeled', file) for file in filenames]
    train_pairs = [(u, l) for u, l in zip(unlabeled_paths, labeled_paths)
                   if os.path.exists(u) and os.path.exists(l)]
    # TODO: add filter to ensure the image width and height is >= 256
    return train_pairs


def normalize(imgpath, maskpath, outshape=(256, 256)):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    img = cv2.imread(imgpath)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    img = cv2.filter2D(img, -1, kernel)
    img = cv2.resize(img, outshape) / 255

    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    mask = cv2.resize(mask, outshape)
    tmpmask = cv2.imread(maskpath, 0)
    tmpmask = cv2.resize(tmpmask, outshape)
    mask = cv2.bitwise_or(mask, tmpmask)
    mask = np.expand_dims(mask, -1) / 255
    return img, mask


def create_dataset(path_pairs):
    img_batch = []
    mask_batch = []
    for imgpath, maskpath in path_pairs:
        img, mask = normalize(imgpath, maskpath)
        img_batch.append(img)
        mask_batch.append(mask)
    return np.array(img_batch), np.array(mask_batch)

In [8]:
def upsample(input, input_size):
    up = Conv2D(input_size, (3, 3), padding='same')(input)
    up = BatchNormalization()(up)
    up = Activation('relu')(up)
    up = Conv2D(input_size, (3, 3), padding='same')(up)
    up = BatchNormalization()(up)
    up = Activation('relu')(up)
    up = Conv2D(input_size, (3, 3), padding='same')(up)
    up = BatchNormalization()(up)
    return Activation('relu')(up)


def get_model(input_shape=(256, 256, 3), num_classes=1):
    inputs = Input(shape=input_shape)
    # 256

    # DOWNSAMPLING to reduce processing time
    down0 = Conv2D(32, (3, 3), padding='same')(inputs)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPool2D((2, 2), strides=(2, 2))(down0)
    # 128

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPool2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPool2D((2, 2), strides=(2, 2))(down2)
    # 32

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPool2D((2, 2), strides=(2, 2))(down3)
    # 16

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4_pool = MaxPool2D((2, 2), strides=(2, 2))(down4)
    # 8

    center = Conv2D(1024, (3, 3), padding='same')(down4_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    # center

    # UPSAMPLING to increase sampling rate
    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = upsample(up4, 512)

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = upsample(up3, 256)

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = upsample(up2, 128)

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = upsample(up1, 64)

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = upsample(up0, 32)

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0)
    model = Model(inputs=inputs, outputs=classify)
    return model

In [9]:
x, y = create_dataset(load_train_pairs())

model_path = "./model_with_weights.hdf5"
checkpoint = ModelCheckpoint(model_path, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=1, mode='min', cooldown=0, min_lr=1e-8)
early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=20)
callbacks = [checkpoint, early, reduce_lr]

model = get_model()
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit(x=x, y=y, epochs=EPOCHS, batch_size=8, validation_split=.5, callbacks=callbacks)

Epoch 1/200
2/2 [==============================] - 12s 2s/step - loss: 0.2129 - mae: 0.4328 - val_loss: 0.9243 - val_mae: 0.9305
Epoch 2/200
2/2 [==============================] - 0s 137ms/step - loss: 0.1431 - mae: 0.3438 - val_loss: 0.9343 - val_mae: 0.9361
Epoch 3/200
2/2 [==============================] - 0s 135ms/step - loss: 0.1093 - mae: 0.2631 - val_loss: 0.9296 - val_mae: 0.9334
Epoch 4/200
2/2 [==============================] - 0s 135ms/step - loss: 0.0858 - mae: 0.2389 - val_loss: 0.9233 - val_mae: 0.9296
Epoch 5/200
2/2 [==============================] - 0s 137ms/step - loss: 0.0828 - mae: 0.2339 - val_loss: 0.9100 - val_mae: 0.9215
Epoch 6/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0786 - mae: 0.2304 - val_loss: 0.8888 - val_mae: 0.9085
Epoch 7/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0766 - mae: 0.2267 - val_loss: 0.8498 - val_mae: 0.8842
Epoch 8/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0718 - mae

2/2 [==============================] - 0s 135ms/step - loss: 0.0660 - mae: 0.2107 - val_loss: 0.0897 - val_mae: 0.2720
Epoch 64/200
2/2 [==============================] - 0s 133ms/step - loss: 0.0660 - mae: 0.2107 - val_loss: 0.0886 - val_mae: 0.2694
Epoch 65/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0647 - mae: 0.2084 - val_loss: 0.0878 - val_mae: 0.2673
Epoch 66/200
2/2 [==============================] - 0s 137ms/step - loss: 0.0641 - mae: 0.2091 - val_loss: 0.0867 - val_mae: 0.2647
Epoch 67/200
2/2 [==============================] - 0s 136ms/step - loss: 0.0635 - mae: 0.2080 - val_loss: 0.0858 - val_mae: 0.2623
Epoch 68/200
2/2 [==============================] - 0s 135ms/step - loss: 0.0644 - mae: 0.2082 - val_loss: 0.0850 - val_mae: 0.2603
Epoch 69/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0644 - mae: 0.2082 - val_loss: 0.0843 - val_mae: 0.2583
Epoch 70/200
2/2 [==============================] - 0s 148ms/step - loss: 0.0653 - mae: 0

Epoch 125/200
2/2 [==============================] - 0s 135ms/step - loss: 0.0670 - mae: 0.2112 - val_loss: 0.0660 - val_mae: 0.1987
Epoch 126/200
2/2 [==============================] - 0s 141ms/step - loss: 0.0641 - mae: 0.2091 - val_loss: 0.0659 - val_mae: 0.1983
Epoch 127/200
2/2 [==============================] - 0s 136ms/step - loss: 0.0635 - mae: 0.2080 - val_loss: 0.0659 - val_mae: 0.1980
Epoch 128/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0644 - mae: 0.2081 - val_loss: 0.0658 - val_mae: 0.1977
Epoch 129/200
2/2 [==============================] - 0s 133ms/step - loss: 0.0635 - mae: 0.2080 - val_loss: 0.0657 - val_mae: 0.1975
Epoch 130/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0625 - mae: 0.2061 - val_loss: 0.0657 - val_mae: 0.1973
Epoch 131/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0635 - mae: 0.2080 - val_loss: 0.0656 - val_mae: 0.1971
Epoch 132/200
2/2 [==============================] - 0s 133ms/step - 

2/2 [==============================] - 0s 133ms/step - loss: 0.0625 - mae: 0.2061 - val_loss: 0.0634 - val_mae: 0.1917
Epoch 187/200
2/2 [==============================] - 0s 133ms/step - loss: 0.0625 - mae: 0.2061 - val_loss: 0.0634 - val_mae: 0.1919
Epoch 188/200
2/2 [==============================] - 0s 137ms/step - loss: 0.0659 - mae: 0.2107 - val_loss: 0.0634 - val_mae: 0.1917
Epoch 189/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0644 - mae: 0.2081 - val_loss: 0.0633 - val_mae: 0.1918
Epoch 190/200
2/2 [==============================] - 0s 134ms/step - loss: 0.0646 - mae: 0.2083 - val_loss: 0.0634 - val_mae: 0.1920
Epoch 191/200
2/2 [==============================] - 0s 140ms/step - loss: 0.0635 - mae: 0.2079 - val_loss: 0.0634 - val_mae: 0.1921
Epoch 192/200
2/2 [==============================] - 0s 135ms/step - loss: 0.0659 - mae: 0.2107 - val_loss: 0.0633 - val_mae: 0.1919
Epoch 193/200
2/2 [==============================] - 0s 142ms/step - loss: 0.0646 -